In [12]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent / "src"))  # Bootstrap path to allow `services` import
from services.env import setup_project
setup_project()

✅ Project path added to sys.path: d:\portfolio_repo\arb_airdrop\src


In [13]:
import pandas as pd
from services.config import DATA_DIR

In [11]:
wallet_features = pd.read_csv(DATA_DIR/"wallet_30_day_balance_snapshot.csv")
filtered_df = pd.read_csv(DATA_DIR/"transfers_within_30days.csv")

In [ ]:

# Count transactions per wallet in 30 days
tx_counts = filtered_df.groupby('wallet').size().reset_index(name='num_transactions_30_days')

# Sum of positive deltas (inflows) and negative deltas (outflows) per wallet
inflows = filtered_df[filtered_df['delta'] > 0].groupby('wallet')['amount'].sum().reset_index(name='total_inflow_30_days')
outflows = filtered_df[filtered_df['delta'] < 0].groupby('wallet')['amount'].sum().reset_index(name='total_outflow_30_days')
# Note: outflows will be negative sums; take absolute value for clarity
outflows['total_outflow_30_days'] = outflows['total_outflow_30_days'].abs()

# Days active: count unique dates per wallet
active_days = filtered_df.groupby('wallet')['date'].nunique().reset_index(name='days_active')

# Merge all features together on wallet
wallet_features = wallet_features.merge(tx_counts, on='wallet', how='left')
wallet_features = wallet_features.merge(inflows, on='wallet', how='left').merge(outflows, on='wallet', how='left')
wallet_features = wallet_features.merge(active_days, on='wallet', how='left')

# Fill NaNs for wallets with no inflow/outflow (e.g., if none, put 0)
wallet_features[['total_inflow_30_days','total_outflow_30_days','num_transactions_30_days','days_active']] = \
    wallet_features[['total_inflow_30_days','total_outflow_30_days','num_transactions_30_days','days_active']].fillna(0)

# Define churn label
wallet_features['churn_label'] = (wallet_features['cumulative_balance_day_30'] == 0).astype(int)

# print("Engineered features:", wallet_features.shape)
# display(wallet_features.head(10))



Engineered features: (100000, 11)


,Unnamed: 0,wallet,cumulative_balance_day_30,wallet_address,claimed_amount,balance_retention_ratio,num_transactions_30_days,total_inflow_30_days,total_outflow_30_days,days_active,churn_label
0,0,0x00000000009a41862f3b2b0c688b7c0d1940511e,0.00000,0x00000000009a41862f3b2b0c688b7c0d1940511e,4250,0.000000,40,4250.00000,4250.0,1,1
1,1,0x0000000000dfd67ffd6c24251348f7c4f933cab4,0.00000,0x0000000000dfd67ffd6c24251348f7c4f933cab4,1750,0.000000,2,1750.00000,1750.0,1,1
2,2,0x0000000000e189dd664b9ab08a33c4839953852c,0.00000,0x0000000000e189dd664b9ab08a33c4839953852c,2250,0.000000,2,2250.00000,2250.0,1,1
3,3,0x000000000279ef217428b1c3906ec8124784b70f,0.00000,0x000000000279ef217428b1c3906ec8124784b70f,3250,0.000000,2,3250.00000,3250.0,1,1
4,4,0x0000000009572a244a6c2d06ffe7be30e3bd2aec,10.00000,0x0000000009572a244a6c2d06ffe7be30e3bd2aec,625,0.016000,3,635.00000,625.0,2,0
5,5,0x000000001786891880872aa0895d9175f99dc9f5,0.00000,0x000000001786891880872aa0895d9175f99dc9f5,1125,0.000000,2,1125.00000,1125.0,1,1
6,6,0x00000000747b9910b5d295b231429d47060c3995,0.00000,0x00000000747b9910b5d295b231429d47060c3995,1750,0.000000,11,1750.00000,1750.0,1,1
7,7,0x000000009fca059dda53d07589298ea6842bd8f9,0.00000,0x000000009fca059dda53d07589298ea6842bd8f9,1750,0.000000,2,1750.00000,1750.0,1,1
8,8,0x00000000e8334445e02d819eafd75addf2133a5f,15.34101,0x00000000e8334445e02d819eafd75addf2133a5f,1750,0.008766,3,1765.34101,1750.0,2,0
9,9,0x0000000114f09a315f2d31d680e236f9b74714ed,10.00000,0x0000000114f09a315f2d31d680e236f9b74714ed,625,0.016000,3,635.00000,625.0,2,0


In [ ]:
wallet_features.to_csv(DATA_DIR/"wallet_features.csv")